In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
cols_req=['Holding Dealer Organic ID',
 'Holding Dealer DESC',
 'Holding Partner Branch DESC',
 'Customer Country NAME',
 'Customer Territory NAME',
 'Customer City ID',
 'Customer ID ID',
 'Alternate ID ID',
 'Customer Termination Date ID',
 'Technician NAME',
 'Panel Type NAME',
 'Panel Description NAME',
 'Modem Serial # ID',
 'Property Type NAME',
 'Sensor Group ID',
 'Sensor Group DESC',
 'Combined Device Type DESC',
 'Device Model Name Name',
 'Device Manufacturer NAME',
 'Device ID',
 'Device Organic ID',
 'Device NAME',
 'Device Firmware Version NAME',
 'Device Unique ID (MAC) ID',
 'Device Install Date ID',
 'Device Removal Date ID15525']

In [ ]:
Inventory_data= pd.read_table('Inventory-20230207.csv',low_memory=False,sep="|",usecols=cols_req)

In [ ]:
Inventory_data.info()

In [ ]:
Inventory_data.head()

In [ ]:
Inventory_data = Inventory_data.rename(columns={'@version': 'version'})

In [ ]:
rep_chars = ' |\|-|:|/|#'

Inventory_data.columns = Inventory_data.columns.str.replace(rep_chars, '_')

In [ ]:
Inventory_data.info(verbose=True)

In [ ]:
Inventory_data = Inventory_data.rename(columns={'Device_Unique_ID_(MAC)_ID': 'Device_Unique_ID_MAC_ID'})

In [ ]:
Inventory_data = Inventory_data.rename(columns={'Device_Removal_Date_ID15525': 'Device_Removal_Date'})

In [ ]:
Inventory_data['Combined_Device_Type_DESC'].value_counts()

In [ ]:
Inventory_data['Customer_Termination_Date_ID'].value_counts()

In [ ]:
Inventory_data['Device_Removal_Date'].value_counts()

In [ ]:
cols_req=['Customer_ID_ID','Combined_Device_Type_DESC','Device_Install_Date_ID',
Inventory_data_1=Inventory_data[

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

Table_BQ = 'ADC_updated.ADC_Inventory_device_data_Feb2023'

bq_table_instance= bq_client.load_table_from_dataframe(Inventory_data, Table_BQ,job_config=config)

In [ ]:
Camera_details_query='''


with data_main as ( select * from (
  SELECT 
    Customer_ID_ID as customer_id,
    combined_device_type_desc,
    Device_ID
    FROM `divgpras-pr-579355.ADC_updated.ADC_Inventory_device_data_Feb2023` 
    where Combined_Device_Type_DESC='Camera'
    and date(Device_Removal_Date) is NULL 

)
PIVOT
(
  count(distinct Device_ID) AS count_devices
  FOR Combined_Device_Type_DESC in ('Camera')
)

)

select * from data_main



'''

In [ ]:
Camera_count_DF=extract_bq_data(bq_client, sql=Camera_details_query)

In [ ]:
Camera_count_DF.info()

In [ ]:
Camera_count_DF.to_csv('SHS_camera_count_Feb2023.csv',index=False)

In [ ]:
Telus_Camera_details=pd.read_csv('adc_camera_device_details_telus_Nov2022.csv',low_memory=False)

In [ ]:
Telus_Camera_details.info()

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

Table_BQ = 'ADC_updated.ADC_Camera_details'

bq_table_instance= bq_client.load_table_from_dataframe(Telus_Camera_details, Table_BQ,job_config=config)